In [ ]:
from applications.nsde_calibration.heston_call import *

In This tutorial, we walk through how to obtain the calibrated stock trajectories using LV and LSV Neural-SDE models, based on the implementation of [Gierjatowicz et. al.](https://arxiv.org/abs/2007.04154). We also explore functionalities in **`heston_call.py`**.

In [ ]:
heston = HestonModel(
        x0=1,  # Initial stock price
        r=0.025,  # Risk-free rate
        V0=0.04, # Initial variance
        kappa=0.78, mu=0.11, eta=0.68, rho=0.044, # Heston parameters
        dt=1/96, timesteps=96, 
        seed=42  # Fixing seed for reproducibility
    )

In [ ]:
    # Simulate stock price paths
n_paths = 5  # Number of paths to plot
stock_paths, var_paths, running_max = heston.Heston_stock(itr=n_paths)

In [ ]:
heston.plot_paths(stock_paths)

In [ ]:
strike_prices = np.arange(0.8, 1.21, 0.02)
maturities = list(range(16, 33, 16))  # 6 months, 1 year
vanilla_prices, exotic_prices = heston.price_option_for_t(
        strikes_call=strike_prices, 
        maturities_in_years=maturities, 
        t_in_years=0.25,  # Pricing at 3 months in
        outer_itr=5, inner_itr=10
    )

In [ ]:
vanilla_prices.shape

In [ ]:
heston.plot_vanilla_prices(maturities,strike_prices, vanilla_prices)

In [ ]:
print("Vanilla Call Option Prices (rows: maturities, columns: strikes):")
print(vanilla_prices)

print("\nExotic Option Prices (maturities):")
print(exotic_prices)

In [ ]:
from src.adapted_mtgl.utils import *

To get Neural-SDE calibrated stock trajectory and the associated variance trajectory, follow the follwing commands:

(1). **`cd ..`** to Navigate first to the outer directroy.

(2). Run the following command for Local Volatility (LV) based Neural-SDE model: 
```
python scripts/run_adapted_LV.py \
    --device 0 \
    --n_layers 4 \
    --vNetWidth 128 \
    --MC_samples_test 5000 \
    --batch_size 1000 \
    --n_epochs 100 \
    --save_dir data/nsde_calibration/
``` 
Follow instructions in **`scripts/run_adapted_LV.py`** for instructions on adding extra customized parameters or arguments.

Run the following command for Local Stochastic Volatility (LSV) based Neural-SDE model: 
```
python scripts/run_adapted_LSV.py \
    --device 0 \
    --n_layers 4 \
    --vNetWidth 128 \
    --MC_samples_test 5000 \
    --batch_size 1000 \
    --n_epochs 100 \
    --save_dir data/nsde_calibration/
```    
Follow instructions in **`scripts/run_adapted_LSV.py`** for instructions on adding extra customized parameters or arguments.

You may also find instructions and argument definitions in **`scripts/argparse_helper.py`** useful.


In [ ]:
traj_txt = "data/nsde_calibration/stock_traj_LSV.txt"
var_txt = "data/nsde_calibration/var_traj_LSV.txt"
traj_mat = load_txt_as_matrix(traj_txt)
print(traj_mat.shape)
var_mat = load_txt_as_matrix(var_txt)
print(var_mat.shape)

In [ ]:
print(traj_mat[0, :10])

In [ ]:
mat_ls, vanilla_payoff_ls = heston.price_payoff_coupling(
    x = traj_mat,
    v = var_mat,
    strikes_call=strike_prices,
    maturities=maturities,
    inner_itr=10
)

In [ ]:
# Print one example result
print("\nPrice-Payoff Coupling for First Maturity and Strike (Outer Path 0, First Time Step):")
print(mat_ls[0][0, 0, 0], " vs. ", vanilla_payoff_ls[0][0, 0])